# Trace Buffer - Tracing SPI Transactions
The Trace_Buffer class can monitor the waveform and transations on PMODA, PMODB, and ARDUINO connectors.

This demo shows how to use this class to track SPI transactions. For this demo, users have to connect the Pmod OLED to PMODB.

## Step 1: Overlay Management
Users have to import all the necessary classes. Make sure to use the right bitstream.

In [1]:
from pprint import pprint
from time import sleep
from pynq import PL
from pynq import Overlay
from pynq.drivers import Trace_Buffer
from pynq.iop import Pmod_OLED
from pynq.iop import PMODA
from pynq.iop import PMODB
from pynq.iop import ARDUINO

ol = Overlay("base.bit")
ol.download()

## Step 2: Instantiating OLED
Although this demo can also be done on PMODA, we use PMODB in this demo.

In [2]:
oled = Pmod_OLED(PMODB)

## Step 3: Tracking Transactions
Instantiating the trace buffer with SPI protocol. The SPI clock is controlled by the 100MHz IO Processor (IOP). The SPI clock period is 16 times the IOP clock rate based on the settings of the IOP SPI controller. Hence we set the sample rate to 20MHz. 

After starting the trace buffer DMA, also start to write some characters. Then stop the trace buffer DMA.

In [3]:
tr_buf = Trace_Buffer(PMODB,pins=[0,1,2,3],probes=['CS','MOSI','NC','CLK'],
                      protocol="spi",rate=20000000)

# Start the trace buffer
tr_buf.start()

# Write characters
oled.write("1 2 3 4 5 6")

# Stop the trace buffer
tr_buf.stop()

## Step 4: Parsing and Decoding Transactions
The trace buffer object is able to parse the transactions into a `*.csv` file (saved into the same folder as this script).
    
Then the trace buffer object can also decode the transactions using the open-source `sigrok` decoders. The decoded file (`*.pd`) is saved into the same folder as this script.

Reference:
https://sigrok.org/wiki/Main_Page

In [4]:
# Configuration for PMODB
start = 25000
stop = 35000

# Parsing and decoding
tr_buf.parse("spi_trace.csv",start,stop)
tr_buf.decode("spi_trace.pd",
              options=':wordsize=8:cpol=0:cpha=0')

## Step 5: Displaying the Result

The final waveform and decoded transactions are shown using the open-source `wavedrom` library.

**Note**: It may take a while for the waveforms to be displayed.

Reference:
https://www.npmjs.com/package/wavedrom

In [5]:
tr_buf.display()